In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import yaml
import imageio
from object_detection.utils import dataset_util

In [2]:
sess=tf.InteractiveSession()

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt

In [5]:
def check_bounding_boxes(img_path,box_yaml):
    boxes =yaml.load(box_yaml)
    imageio.imread(img_path)
    

In [6]:
im_data_raw = imageio.imread('/home/amol/TrafficLightImages/train/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/207458.png')
im_data = tf.cast(im_data_raw,tf.float32)
plt.figure(figsize=(200,100))
plt.imshow(im_data_raw)

In [59]:
im_data = im_data.eval()

In [63]:
im_data = tf.reshape(im_data,[1,720,1280,3]).eval()

In [4]:
import yaml
boxes = yaml.load(open('/home/amol/TrafficLightImages/train/train.yaml','r'))

In [7]:
box=boxes[4]

In [3]:
test_boxes=yaml.load(open('/home/amol/TrafficLightImages/test/test.yaml','r'))

In [5]:
label_set = set()
for box in boxes:
    for boxi in box['boxes']:
        label_set.add(boxi['label'].encode())

In [54]:
label_set

{b'Green',
 b'GreenLeft',
 b'GreenRight',
 b'GreenStraight',
 b'GreenStraightLeft',
 b'GreenStraightRight',
 b'Red',
 b'RedLeft',
 b'RedRight',
 b'RedStraight',
 b'RedStraightLeft',
 b'Yellow',
 b'off'}

In [4]:
test_label_set = set()
for box in test_boxes:
    for boxi in box['boxes']:
        test_label_set.add(boxi['label'].encode())

In [5]:
test_label_set

{b'Green', b'Red', b'Yellow', b'off'}

In [70]:
img_with_boxes = tf.image.draw_bounding_boxes(im_data,tf.reshape([[box['y_min']/720,box['x_min']/1280,box['y_max']/720,box['x_max']/1280] for box in box['boxes']],[1,3,4])).eval()

In [78]:
new_img = tf.cast(img_with_boxes,tf.uint8)

In [9]:
#plt.figure(figsize=(200,100))
#plt.imshow(new_img.eval()[0])

In [6]:
import numpy as np

In [7]:
def validate_bbox(box_info):
    vec = [box_info['x_max'],box_info['x_min'],box_info['y_max'],box_info['y_min']]
    check1 = any([(not np.isnan(i)) and (i>0) for i in vec])
    check2 = (box_info['x_max']<=1280) and (box_info['x_min']<=1280) and (box_info['x_max']>=box_info['x_min'])
    check3 = (box_info['y_max']<=720) and (box_info['y_min']<720) and (box_info['y_max']>=box_info['y_min'])
    return check1 and check2 and check3

In [8]:
def extract_box_info(boxes_dict):
    result_box_info = []
    labels = []
    for box_info in boxes_dict:
        if validate_bbox(box_info):
            result_box_info.append(np.array([box_info['x_max'],box_info['x_min'],box_info['y_max'],box_info['y_min']]))
            labels.append(box_info['label'].encode())
        else:
            return None 
    result_box_info = np.array(result_box_info)
    if(len(labels)==0):
        return {'x_max':np.array([],dtype=np.float32),'x_min':np.array([],dtype=np.float32),'y_min':np.array([],dtype=np.float32),'y_max':np.array([],dtype=np.float32),'labels':labels}
    return {'x_max':result_box_info[:,0],'x_min':result_box_info[:,1],'y_max':result_box_info[:,2],'y_min':result_box_info[:,3],'labels':labels}


In [6]:
[a = extract_box_info(boxes[4]['boxes'])]

In [11]:
nothing = [extract_box_info(box['boxes']) for box in boxes]

> <ipython-input-7-cf0dd8c52669>(4)extract_box_info()
-> for box_info in boxes_dict:
{'label': 'Red', 'occluded': False, 'x_max': 1282.5703578205, 'x_min': 1267.8285210866, 'y_max': 269.4957199198, 'y_min': 220.4761974741}
> <ipython-input-7-cf0dd8c52669>(4)extract_box_info()
-> for box_info in boxes_dict:
{'label': 'Green', 'occluded': False, 'x_max': 785.1169839057, 'x_min': 786.2394607942, 'y_max': 286.8550689898, 'y_min': 287.1045082984}


BdbQuit: 

In [70]:
np.array([],dtype=np.float32)

array([], dtype=float32)

In [9]:
LABELMAP = dict(zip(list(label_set),range(1,1+len(label_set))))#not sure if id should start from one but just to be safe

In [10]:
LABELMAP

{b'Green': 5,
 b'GreenLeft': 13,
 b'GreenRight': 4,
 b'GreenStraight': 8,
 b'GreenStraightLeft': 2,
 b'GreenStraightRight': 7,
 b'Red': 11,
 b'RedLeft': 6,
 b'RedRight': 1,
 b'RedStraight': 10,
 b'RedStraightLeft': 9,
 b'Yellow': 12,
 b'off': 3}

In [11]:
# code is modified from the tensorflow object detection github
def create_tf_example(filename,boxes):
    """Creates a tf.Example proto from sample cat image.

    Args:
        encoded_image_data: The jpg encoded data of the cat image.

    Returns:
        example: The created tf.Example.
    """

    height = 720
    width = 1280
    image_format = b'png'

    xmins = np.array([i/width for i in boxes['x_min']])
    xmaxs = np.array([i/width for i in boxes['x_max']])
    ymins = np.array([i/height for i in boxes['y_min']])
    ymaxs = np.array([i/height for i in boxes['y_max']])
    classes_text = boxes['labels']
    classes = [LABELMAP[i] for i in classes_text]
    with tf.gfile.GFile(filename,'rb') as fid:
        encoded_image_data = fid.read()
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename.encode()),
      'image/source_id': dataset_util.bytes_feature(filename.encode()),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [12]:
import contextlib2
from object_detection.dataset_tools import tf_record_creation_util

num_shards=10
output_filebase='/home/amol/ObjectDetection/data/train_'
filename = '/home/amol/TrafficLightImages/train/train.yaml'
writer = tf.python_io.TFRecordWriter
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
      tf_record_close_stack, output_filebase, num_shards)
    all_image_boxes = yaml.load(open(filename,'r'))
    index = 0
    for box_dict in all_image_boxes:
        boxes_per_image = extract_box_info(box_dict['boxes'])
        if boxes_per_image:
            path = '/home/amol/TrafficLightImages/train/'+box_dict['path']
            tf_example = create_tf_example(path,boxes_per_image)
            output_shard_index = index % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
            index += 1

    
    
    

In [60]:
t=create_tf_example('/home/amol/TrafficLightImages/train/'+boxes[4]['path'],extract_box_info(boxes[4]['boxes']))

In [13]:
#bosch test files have wrong relative paths in yaml file so need a different setup to write test TF records
output_filebase='/home/amol/ObjectDetection/data/test_'
filename = '/home/amol/TrafficLightImages/test/test.yaml'
writer = tf.python_io.TFRecordWriter
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
      tf_record_close_stack, output_filebase, num_shards)
    all_image_boxes = yaml.load(open(filename,'r'))
    index = 0
    for box_dict in all_image_boxes:
        boxes_per_image = extract_box_info(box_dict['boxes'])
        if boxes_per_image:
            fixed_path = '/home/amol/TrafficLightImages/test/rgb/test/'+ box_dict['path'].split('/')[-1]
            tf_example = create_tf_example(fixed_path,boxes_per_image)
            output_shard_index = index % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
            index += 1

In [79]:
'/home/amol/TrafficLightImages/test/rgb/test/38358.png'.split('/')[-1]

'38358.png'

In [87]:
with open('/home/amol/ObjectDetection/data/label_map.pbtxt','w') as fid:
    for key in LABELMAP.keys():
        output_string = "item{{id:{id}\n name:'{name}' }}"
        fid.write(output_string.format(id=LABELMAP[key],name=key.decode('utf-8')))
        fid.write('\n\n')
    
    

In [86]:
len(LABELMAP)

13